# Magniber 버전 1 복호화 절차

Magniber 랜섬웨어 암호기능 분석 보고서 (https://seed.kisa.or.kr/kisa/Board/56/detailView.do) 문서 기준으로 복호화 절차를 수행,

Written by DarkLion (어둠사자)

In [55]:
# 필요 라이브러리 불러오기
import os
from Crypto.Cipher import AES

In [56]:
# 작업 디렉토리(폴더) 지정
root_directory = '.'

# 키쌍 (IV, KEY) 지정
# 참고: 암호화 당시 사용된 key가 hardcoded key(샘플 내장 키)가 아니라
#        network key(Tor 네트워크에서 받은 키)인 경우 유효한 복호화를 진행할 수 없음
keypairs = {
    'kgpvwnr': { 'iv': 'N3ii3Ne9O10VC5LU', 'key': 'tcHa16q59aBeqt65' },   # kgpvwnr
    'ymdmf':   { 'iv': 'CYr41IC975wBjaP0', 'key': 'L24h7p25Tu627X77' }    # ymdmf
}

In [57]:
# 복호화 진행 (AES-128, CBC 모드)
def Decrypt_Magniber(iv, key, suffix, filepath, filesize):
    readfile = open(filepath, 'rb')
    writefile = open(filepath.split('.' + suffix)[0], 'wb')

    readfile.seek(16)
    cipher = readfile.read(filesize - 16)
    
    b_iv = iv.encode("utf8")
    b_key = key.encode("utf8")
    
    print("IV:", '0x' + b_iv.hex())
    print("Key:", '0x' + b_key.hex())

    encryptor = AES.new(b_key, AES.MODE_CBC, b_iv)
    padding_plain = encryptor.decrypt(cipher) # 복호화 된 평문 (패딩 포함)
    size = filesize - 16 - padding_plain[-1] # 패딩 전 원본 파일 크기
    plain = padding_plain[0:size] # 복호화 된 평문 (패딩 제외)
    #plain = padding_plain
    
    writefile.write(plain)

    readfile.close()
    writefile.close()

In [58]:
# 디렉토리 내 파일 탐색
for root, dirs, files in os.walk(root_directory):
    for file in files:
        suffix = file.split(".")[-1]    # 확장자 확인
        if suffix in keypairs:          # 확장자와 일치하는 키쌍(keypair)이 있는지 확인
            filepath = os.path.join(root_directory, file)
            print('Try decrypting file... ', filepath)
            Decrypt_Magniber(keypairs[suffix]['iv'], keypairs[suffix]['key'], suffix, filepath, os.path.getsize(filepath))
            print('Done.')
            print()

Try decrypting file...  .\IMG309.jpg.kgpvwnr
IV: 0x4e336969334e65394f31305643354c55
Key: 0x74634861313671353961426571743635
Done.

Try decrypting file...  .\IMG310.jpg.kgpvwnr
IV: 0x4e336969334e65394f31305643354c55
Key: 0x74634861313671353961426571743635
Done.

